<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/chat_engine/chat_engine_react.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# 聊天引擎 - ReAct代理模式


ReAct是一个基于代理的聊天模式，构建在对您的数据进行查询的引擎之上。


对于每次聊天互动，代理程序会进入一个ReAct循环：
* 首先决定是否使用查询引擎工具，并提出适当的输入
* （可选）使用查询引擎工具并观察其输出
* 决定是重复还是给出最终回应


这种方法很灵活，因为它可以灵活地选择是否查询知识库。
然而，性能也更多地取决于LLM的质量。
您可能需要更多的强制措施，以确保它在正确的时机选择查询知识库，而不是产生幻觉式的答案。


如果您在colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
%pip install llama-index-llms-anthropic
%pip install llama-index-llms-openai

In [ ]:
!pip install llama-index

## 下载数据


In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

## 五行代码即可开始


加载数据并构建索引


In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from llama_index.llms.anthropic import Anthropic

llm = OpenAI()
data = SimpleDirectoryReader(input_dir="./data/paul_graham/").load_data()
index = VectorStoreIndex.from_documents(data)

配置聊天引擎


In [ ]:
chat_engine = index.as_chat_engine(chat_mode="react", llm=llm, verbose=True)

与您的数据交谈


In [ ]:
response = chat_engine.chat(
    "Use the tool to answer what did Paul Graham do in the summer of 1995?"
)

Thought: I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'What did Paul Graham do in the summer of 1995?'}
Observation: 
In the summer of 1995, Paul Graham worked on building a web application for making web applications. He recruited Dan Giffin, who had worked for Viaweb, and two undergrads who wanted summer jobs, and they got to work trying to build what it's now clear is about twenty companies and several open source projects worth of software. The language for defining applications would of course be a dialect of Lisp.
Response: In the summer of 1995, Paul Graham worked on building a web application for making web applications. He recruited Dan Giffin, who had worked for Viaweb, and two undergrads who wanted summer jobs, and they got to work trying to build what it's now clear is about twenty companies and several open source projects worth of software. The language for defining applications would of course be a dialect of Lisp

In [ ]:
print(response)

In the summer of 1995, Paul Graham worked on building a web application for making web applications. He recruited Dan Giffin, who had worked for Viaweb, and two undergrads who wanted summer jobs, and they got to work trying to build what it's now clear is about twenty companies and several open source projects worth of software. The language for defining applications would of course be a dialect of Lisp.


### 自定义LLM


使用Anthropic（“claude-2”）


In [ ]:
llm = Anthropic()

配置聊天引擎


In [ ]:
chat_engine = index.as_chat_engine(llm=llm, chat_mode="react", verbose=True)

In [ ]:
response = chat_engine.chat("what did Paul Graham do in the summer of 1995?")

Thought: I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'what did Paul Graham do in the summer of 1995?'}
Observation:  Based on the context, in the summer of 1995 Paul Graham:

- Painted a second still life using the same objects he had used for a previous still life painting.

- Looked for an apartment to buy in New York, trying to find a neighborhood similar to Cambridge, MA. 

- Realized there wasn't really a "Cambridge of New York" after visiting the actual Cambridge.

The passage does not mention what Paul Graham did in the summer of 1995 specifically. It talks about painting a second still life at some point and looking for an apartment in New York at some point, but it does not connect those events to the summer of 1995.
Response: The passage does not provide enough information to know specifically what Paul Graham did in the summer of 1995. It mentions some activities like painting and looking for an apartment in New York

In [ ]:
print(response)

The passage does not provide enough information to know specifically what Paul Graham did in the summer of 1995. It mentions some activities like painting and looking for an apartment in New York, but does not say these occurred specifically in the summer of 1995.


In [ ]:
response = chat_engine.chat("What did I ask you before?")

Response:  You asked me "what did Paul Graham do in the summer of 1995?".


In [ ]:
print(response)

 You asked me "what did Paul Graham do in the summer of 1995?".


把聊天引擎重置


In [ ]:
chat_engine.reset()

In [ ]:
response = chat_engine.chat("What did I ask you before?")

Response: I'm afraid I don't have any context about previous questions in our conversation. This seems to be the start of a new conversation between us.


In [ ]:
print(response)

I'm afraid I don't have any context about previous questions in our conversation. This seems to be the start of a new conversation between us.
